In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [9]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [12]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.SGD(model.parameters(), lr=2e-2)

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.1778826117515564
epoch:  1, loss: 0.16626380383968353
epoch:  2, loss: 0.155645951628685
epoch:  3, loss: 0.14594502747058868
epoch:  4, loss: 0.13708345592021942
epoch:  5, loss: 0.12899428606033325
epoch:  6, loss: 0.12160565704107285
epoch:  7, loss: 0.11485477536916733
epoch:  8, loss: 0.10868828743696213
epoch:  9, loss: 0.10305129736661911
epoch:  10, loss: 0.09789412468671799
epoch:  11, loss: 0.09317369014024734
epoch:  12, loss: 0.08884841948747635
epoch:  13, loss: 0.08488387614488602
epoch:  14, loss: 0.08124840259552002
epoch:  15, loss: 0.07791347056627274
epoch:  16, loss: 0.07485339045524597
epoch:  17, loss: 0.07204516977071762
epoch:  18, loss: 0.06946806609630585
epoch:  19, loss: 0.06710252910852432
epoch:  20, loss: 0.06493081897497177
epoch:  21, loss: 0.06293690204620361
epoch:  22, loss: 0.0611058734357357
epoch:  23, loss: 0.05942436680197716
epoch:  24, loss: 0.05788009613752365
epoch:  25, loss: 0.05646181106567383
epoch:  26, loss: 0.055159

In [13]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = -6493.385020554838
Test metrics:  R2 = -5726.414842743044


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=1e-2)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().cpu().numpy().item()))

epoch:  0, loss: 0.2209373116493225
epoch:  1, loss: 0.18289239704608917
epoch:  2, loss: 0.1485273241996765
epoch:  3, loss: 0.11472073942422867
epoch:  4, loss: 0.08132372051477432
epoch:  5, loss: 0.05237060412764549
epoch:  6, loss: 0.037233103066682816
epoch:  7, loss: 0.049321211874485016
epoch:  8, loss: 0.06692074984312057
epoch:  9, loss: 0.06430836766958237
epoch:  10, loss: 0.052060727030038834
epoch:  11, loss: 0.041838426142930984
epoch:  12, loss: 0.037303145974874496
epoch:  13, loss: 0.03734647110104561
epoch:  14, loss: 0.03955647721886635
epoch:  15, loss: 0.04197460412979126
epoch:  16, loss: 0.04361793026328087
epoch:  17, loss: 0.044207241386175156
epoch:  18, loss: 0.043829191476106644
epoch:  19, loss: 0.04272680729627609
epoch:  20, loss: 0.04120100289583206
epoch:  21, loss: 0.03956182301044464
epoch:  22, loss: 0.03809279948472977
epoch:  23, loss: 0.03701317310333252
epoch:  24, loss: 0.03644777089357376
epoch:  25, loss: 0.036399804055690765
epoch:  26, loss

In [8]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.6688919425876207
Test metrics:  R2 = 0.5650632959382436
